In [1]:
!git clone https://github.com/1hamzaiqbal/MFCLIP_acv.git

Cloning into 'MFCLIP_acv'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (422/422), done.
remote: Compressing objects: 100% (317/317), done.
remote: Total 422 (delta 89), reused 418 (delta 88), pack-reused 0 (from 0)
Receiving objects: 100% (422/422), 3.43 MiB | 25.45 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [5]:
%%shell
cd MFCLIP_acv/
mkdir -p logs/baseline/
chmod +x ./scripts/baseline.sh
./scripts/baseline.sh

CalledProcessError: Command 'cd MFCLIP_acv/
mkdir -p logs/baseline/
chmod +x ./scripts/baseline.sh
./scripts/baseline.sh
' returned non-zero exit status 1.